In [ ]:
import os
import sys
from pathlib import Path

In [ ]:
# HOMEディレクトリ設定(環境に合わせて変更してください)
if "google.colab" in sys.modules:
    # Google Colaboratoryの場合
    HOME = Path("/content/drive/MyDrive/signate/NEDOG")

    # Google Driveをマウント
    if not os.path.exists("/content/drive"):
        from google.colab import drive
        drive.mount("/content/drive")
else:
    # それ以外
    HOME = Path("..")

# INPUT/WORKINGディレクトリ設定
INPUT = HOME / "input"
WORKING = HOME / "working"

In [ ]:
import warnings
warnings.simplefilter('ignore', FutureWarning)

In [ ]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import scipy.signal

# データ読み取り

In [ ]:
# スムージング関数
def smoothing(df, col):
    icol = df.columns.get_loc(col)
    for i in range(len(df)//30):
        df.iloc[i*30:(i+1)*30,icol] = scipy.signal.savgol_filter(
            df.iloc[i*30:(i+1)*30,icol].values,9,1)

In [ ]:
# 速度推論結果
tr_pred_vel_df = pd.read_pickle(WORKING / 'tr_pred_vel_stacking.pickle')
tr_pred_vel_df = tr_pred_vel_df.iloc[:len(tr_pred_vel_df)//2]
ts_pred_vel_df = pd.read_pickle(WORKING / 'ts_pred_vel_stacking.pickle')
ts_pred_vel_df = ts_pred_vel_df.iloc[:len(ts_pred_vel_df)//2]

In [ ]:
# 直交座標系変換関数
def conv_rectangular(df):
    x = df['pred_vel_r'] * np.cos(df['pred_vel_e']) * df['pred_vel_c']
    y = df['pred_vel_r'] * np.cos(df['pred_vel_e']) * df['pred_vel_s']
    z = df['pred_vel_r'] * np.sin(df['pred_vel_e'])
    return x, y, z

In [ ]:
# 極座標を直交座標系に変換し平均をとる
x, y, z = conv_rectangular(tr_pred_vel_df)
tr_pred_vel_df['pred_vel_x'] = (tr_pred_vel_df['pred_vel_x'] + x) / 2
tr_pred_vel_df['pred_vel_y'] = (tr_pred_vel_df['pred_vel_y'] + y) / 2
tr_pred_vel_df['pred_vel_z'] = (tr_pred_vel_df['pred_vel_z'] + z) / 2
tr_pred_vel_df.drop(
    ['pred_vel_r','pred_vel_s','pred_vel_c','pred_vel_e'], axis=1, inplace=True
)
x, y, z = conv_rectangular(ts_pred_vel_df)
ts_pred_vel_df['pred_vel_x'] = (ts_pred_vel_df['pred_vel_x'] + x) / 2
ts_pred_vel_df['pred_vel_y'] = (ts_pred_vel_df['pred_vel_y'] + y) / 2
ts_pred_vel_df['pred_vel_z'] = (ts_pred_vel_df['pred_vel_z'] + z) / 2
ts_pred_vel_df.drop(
    ['pred_vel_r','pred_vel_s','pred_vel_c','pred_vel_e'], axis=1, inplace=True
)

In [ ]:
# CV用
tr_vel_df = pd.read_pickle(WORKING / 'prep2_tr_vel.pickle')
tr_vel_df = tr_vel_df.iloc[:len(tr_vel_df)//2]
tr_vel_df = pd.concat([
    tr_vel_df, tr_pred_vel_df,
], axis=1)

smoothing(tr_vel_df, 'pred_vel_x')
smoothing(tr_vel_df, 'pred_vel_y')
smoothing(tr_vel_df, 'pred_vel_z')

In [ ]:
# submit用
ts_vel_df = pd.read_pickle(WORKING / 'prep2_ts_emg.pickle')
ts_vel_df = ts_vel_df.iloc[:len(ts_vel_df)//2]
ts_vel_df = pd.concat([
    ts_vel_df[['subject','trial','time']], ts_pred_vel_df,
], axis=1)

smoothing(ts_vel_df, 'pred_vel_x')
smoothing(ts_vel_df, 'pred_vel_y')
smoothing(ts_vel_df, 'pred_vel_z')

# CV

In [ ]:
s_rmse = []
for subject in range(1,1+tr_vel_df['subject'].max()):
    fs = tr_vel_df['subject'] == subject
    t_rmse = []
    for trial in range(1, 1+tr_vel_df.loc[fs,'trial'].max()):
        ft = tr_vel_df['trial'] == trial
        dx = tr_vel_df.loc[fs & ft, 'vel_x'].values - tr_vel_df.loc[fs & ft, 'pred_vel_x'].values
        dy = tr_vel_df.loc[fs & ft, 'vel_y'].values - tr_vel_df.loc[fs & ft, 'pred_vel_y'].values
        dz = tr_vel_df.loc[fs & ft, 'vel_z'].values - tr_vel_df.loc[fs & ft, 'pred_vel_z'].values
        rmse = np.sqrt(np.mean(dx**2 + dy**2 + dz**2))
        t_rmse.append(rmse)
    s_rmse.append(np.mean(t_rmse))
print(s_rmse)
print((np.mean(s_rmse[:4])+s_rmse[4]+s_rmse[5])/3)

# submission生成

In [ ]:
# 回転変換
def rotate_vel(df, angle, xcol='pred_vel_x', ycol='pred_vel_y'):
    rot = np.array(
        [[np.cos(angle),-np.sin(angle)],[np.sin(angle),np.cos(angle)]]
    )
    vel_x, vel_y = np.dot(rot, [df[xcol].values, df[ycol].values])
    return vel_x, vel_y

In [ ]:
# 軸補正逆変換
theta = {
    1: 0.3490504635557099,
    2: 0.38184457596775556,
    3: 0.5358451499036176,
    4: 0.08057051473870801,
    5: 0.3388934406953976
}
for subject in range(1,1+ts_vel_df['subject'].max()):
    f = ts_vel_df['subject'] == subject
    vel_x, vel_y = rotate_vel(ts_vel_df[f], theta[subject])
    ts_vel_df.loc[f, 'pred_vel_x'] = vel_x
    ts_vel_df.loc[f, 'pred_vel_y'] = vel_y

In [ ]:
# sample submit読み取り
with open(INPUT / 'sample_submit.json') as f:
    sample_submit = json.load(f)

In [ ]:
# 更新
for subject in range(1, 1+ts_vel_df['subject'].max()):
    fs = ts_vel_df['subject'] == subject
    for trial in range(1, 1+ts_vel_df.loc[fs,'trial'].max()):
        ft = ts_vel_df['trial'] == trial
        vel_x = ts_vel_df.loc[fs & ft, 'pred_vel_x'].values
        vel_y = ts_vel_df.loc[fs & ft, 'pred_vel_y'].values
        vel_z = ts_vel_df.loc[fs & ft, 'pred_vel_z'].values
        for time in range(30):
            sample_submit[f'sub{subject}'][f'trial{trial}'][time][0] = vel_x[time]
            sample_submit[f'sub{subject}'][f'trial{trial}'][time][1] = vel_y[time]
            sample_submit[f'sub{subject}'][f'trial{trial}'][time][2] = vel_z[time]

In [ ]:
# 保存
with open(WORKING / 'submission.json','w') as f:
    json.dump(sample_submit,f)